In [1]:
'''COPY PASTED CORE ON 01/02/2025'''
'''UPDATE AS NEEDED'''
import numpy as np
from numpy import random as npr
cooperate = np.array( [ 1, 0 ] )
defect = np.array( [ 0, 1 ] )
# defining all strategies
# # template function
# parameter_list = [ k, ]
# def strategy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
#     # move either cooperate or defect
#     move
#     return move 

# nice guy function
def nice_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
    move = cooperate
    return move 

# bad guy function
def bad_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
    move = defect
    return move 

# mainly nice guy function
def mainly_nice( parameter_list = [ 0.4 ], round_number = 0, match_history = [[]], player_index = 0 ) : 
    k = parameter_list[ 0 ]
    if k > 0.5 : 
        print( "Wrong function called: this is mainly_nice" )
        return - 1
    u = npr.random( 1 )
    if ( u < k ) : 
        move = defect
    else : 
        move = cooperate
    return move

# mainly bad guy function
def mainly_bad( parameter_list = [ 0.6 ], round_number = 0, match_history = [[]], player_index = 0 ) : 
    k = parameter_list[ 0 ]
    if k < 0.5 : 
        print( "Wrong function called: this is mainly_bad" )
        return - 1
    u = npr.random( 1 )
    if ( u < k ) : 
        move = defect
    else : 
        move = cooperate
    return move

# tit_for_tat function
def tit_for_tat( parameter_list = [ ], round_number = 0, match_history = [[]], player_index = 0 ) : 
    if ( round_number == 0 ) : 
        move = cooperate
    else : 
        move = match_history[ round_number - 1 ][ 1 - player_index ]
    return move

# resentful guy function
def resentful_guy( parameter_list = [ ], round_number = 0, match_history = [[]], player_index = 0 ) : 
    move = cooperate
    for res in match_history :
        if np.all( res[ 1 - player_index  ] == defect ) :
            move = defect
    return move

# trusting guy (increasing odds of cooperating according to match history)
def trusting_guy( parameter_list = [ 0.5 ], round_number = 0, match_history = [[]], player_index = 0 ) :
    # starting trust
    k = parameter_list[ 0 ]
    # environmental trust
    if ( round_number > 0 ) :
        coop = 0. 
        for res in match_history : coop += res[ 1 - player_index  ][ 1 ]
        k = coop / len( match_history )
    u = npr.random( 1 )
    if ( u > k ) : 
        move = defect
    else : 
        move = cooperate
    return move

# balancing guy (cooperates only for an even cooperation history)
def balancing_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) :
    move = defect
    if ( round_number > 0 ) :
        # total cooperation seen
        coop = 0
        for res in match_history : coop += ( res[ 0 ][ 1 ] + res[ 1 ][ 1 ])
        if ( ( coop % 2 ) == 0 ) : 
            move = cooperate
    return move

# dictionary of all strategies
strategies = {
    'NiceGuy' : nice_guy,
    'BadGuy' : bad_guy,
    'MainlyNice' : mainly_nice,
    'MainlyBad' : mainly_bad,
    'TitForTat' : tit_for_tat,
    'ResentfulGuy' : resentful_guy,
    'TrustingGuy' : trusting_guy,
    'Thanos' : balancing_guy,
}

# default Payoff matrix
default_R = 2
default_S = 0
default_T = 3
default_P = 1
Payoff = np.array( [ [ default_R , default_S ], [ default_T, default_P ] ] )

# function to play one round 1 vs 1 for two different* strategies players
def round( key_1, key_2, parameter_list_1, parameter_list_2, round_number, match_history, M = Payoff ) :
    # getting strategies from dictionary
    S_1 = strategies[ key_1 ]
    S_2 = strategies[ key_2 ]
    # computing next move for each player
    u_1 = S_1( parameter_list_1, round_number, match_history, 0 )
    u_2 = S_2( parameter_list_2, round_number, match_history, 1 )
    # computing rewards for each player
    r_1 = np.dot( u_1, Payoff.dot(u_2))
    r_2 = np.dot( u_2, Payoff.dot(u_1))
    # updating match history
    match_history.append( [ u_1, u_2 ] )
    # returning results
    return r_1, r_2

# function to play a match of N rounds 1 vs 1 for two different* strategies player
def match( key_1, key_2, parameter_list_1, parameter_list_2, N_rounds = 10, M = Payoff ) : 
    # match history strarting empty
    history = [ ]
    # starting total rewards
    R_1 = 0
    R_2 = 0
    # list of partial sums
    Rewards = []
    for round_i in range( 0, N_rounds ) :
        r_1, r_2 = round( key_1, key_2, parameter_list_1, parameter_list_2, round_i, history, M )
        R_1 += r_1
        R_2 += r_2
        Rewards.append( [ R_1, R_2 ] )
    return Rewards


In [2]:
# Mattia
# I'll explore the ways to get the combinations of 2 players out of N partecipating in the tourney

# This default package initially seems nice: check documentantion here
# https://docs.python.org/3/library/itertools.html#itertools.combinations
import itertools

# We already got a problem: when using a dict, keys are unique, therefore if there are multiple players with the same type
# a dictionary of this type is not good
test_t = {
    'NiceGuy' : nice_guy,
    'BadGuy' : bad_guy,
    'MainlyNice' : mainly_nice,
    'BadGuy' : bad_guy,
    'BadGuy' : bad_guy,
    'NiceGuy' : nice_guy,
    }

# This only works if all the labels are different from each-other, which means only one player per type
print("If only 1 player per type is present, this works, howeverer it has poor generalization: \n")
a = itertools.combinations(strategies, 2)
for item in a: print(item)
print('\n')

# Example in the case of repeated keys
print("Repeated keys from dictionary do not work example: \n")
a = itertools.combinations(test_t, 2)
for item in a: print(item)
print('\n')

# This is not good: elements that appear only one time in the dict would go against themselves
print("Repeated keys using combination with replacement leads to having matches with oneself(see MainlyNice): \n")
a = itertools.combinations_with_replacement(test_t, 2)
for item in a: print(item)
print('\n')

# It seems the iterabole object is consumed after the cycle
print("Iterable obejct emptied after use: \n")
a_l = [x for x in a]
print(a_l)
print('\n')

# In order to avoid losing the object, a cast to a container is needed, e.g. a list
print("Proper way to use itertools: cast its output as a list in order not to lose it: \n")
a_list = list(itertools.combinations_with_replacement(test_t, 2))
print(a_list[0])
print(a_list[1])
print('\n')

# Original list is not modified
print("Original list is not modified: \n")
print(test_t)
print('\n')

# Therefore, this seems the best way to do it
print("Proper use example: \n")
test_list = ["NiceGuy","BadGuy","MainlyNice","BadGuy","BadGuy","NiceGuy"]
print(f"Starting from this list {test_list} \n")
b = list(itertools.combinations(test_list, 2))
print("Item print in for loop: \n")
for item in b: print(item)
print("\nWe can access the individual element with [0] for 'Player1' and [1] for 'Player2': \n")
for item in b: print(f"{item[0]}  {item[1]}")
print('\n')

If only 1 player per type is present, this works, howeverer it has poor generalization: 

('NiceGuy', 'BadGuy')
('NiceGuy', 'MainlyNice')
('NiceGuy', 'MainlyBad')
('NiceGuy', 'TitForTat')
('NiceGuy', 'ResentfulGuy')
('NiceGuy', 'TrustingGuy')
('NiceGuy', 'Thanos')
('BadGuy', 'MainlyNice')
('BadGuy', 'MainlyBad')
('BadGuy', 'TitForTat')
('BadGuy', 'ResentfulGuy')
('BadGuy', 'TrustingGuy')
('BadGuy', 'Thanos')
('MainlyNice', 'MainlyBad')
('MainlyNice', 'TitForTat')
('MainlyNice', 'ResentfulGuy')
('MainlyNice', 'TrustingGuy')
('MainlyNice', 'Thanos')
('MainlyBad', 'TitForTat')
('MainlyBad', 'ResentfulGuy')
('MainlyBad', 'TrustingGuy')
('MainlyBad', 'Thanos')
('TitForTat', 'ResentfulGuy')
('TitForTat', 'TrustingGuy')
('TitForTat', 'Thanos')
('ResentfulGuy', 'TrustingGuy')
('ResentfulGuy', 'Thanos')
('TrustingGuy', 'Thanos')


Repeated keys from dictionary do not work example: 

('NiceGuy', 'BadGuy')
('NiceGuy', 'MainlyNice')
('BadGuy', 'MainlyNice')


Repeated keys using combination with r

In [3]:
# Check if the number of iterations is as expected by combination calculus
N = len(test_list)
k = 2
from scipy.special import comb
couples = int(comb(N,k))
obtained_couples = len(b)
print(f"Expected couples: {couples};   Obtained couples: {obtained_couples}")
# So the syntax is b[i][j] where i is the ordered number of the couple and j is the index for the player: 0 for player1, 1 for player2
print("Example b[0]: ", b[0])
print("Example b[0][0]: ", b[0][0])
print("Example b[0][1]: ", b[0][1])

Expected couples: 15;   Obtained couples: 15
Example b[0]:  ('NiceGuy', 'BadGuy')
Example b[0][0]:  NiceGuy
Example b[0][1]:  BadGuy


In [5]:
# Initialize from a list of frequencies
freq = np.arange(0,len(strategies),1)
player_partecipants = { x:i for x,i in zip(strategies.keys(),freq) }
player_list = []
print(player_partecipants, '\n')

for item in (list(player_partecipants.keys())):
    for i in range (player_partecipants[item]):
        player_list.append(item)
print(player_list, "\n")

# Initialize from a dictionary of frequencies
player_partecipants_dict = {
    'NiceGuy' : 0,
    'BadGuy' : 1,
    'MainlyNice' : 2,
    'MainlyBad' : 3,
    'TitForTat' : 4,
    'ResentfulGuy' : 5,
    'TrustingGuy' : 6,
    'Thanos' : 7,
}

player_list_from_dict = []
for item in (list(player_partecipants_dict.keys())):
    for i in range (player_partecipants_dict[item]):
        player_list_from_dict.append(item)
print(player_list_from_dict)

print("Are the two methods equivalent? ",player_list==player_list_from_dict)

{'NiceGuy': 0, 'BadGuy': 1, 'MainlyNice': 2, 'MainlyBad': 3, 'TitForTat': 4, 'ResentfulGuy': 5, 'TrustingGuy': 6, 'Thanos': 7} 

['BadGuy', 'MainlyNice', 'MainlyNice', 'MainlyBad', 'MainlyBad', 'MainlyBad', 'TitForTat', 'TitForTat', 'TitForTat', 'TitForTat', 'ResentfulGuy', 'ResentfulGuy', 'ResentfulGuy', 'ResentfulGuy', 'ResentfulGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'Thanos', 'Thanos', 'Thanos', 'Thanos', 'Thanos', 'Thanos', 'Thanos'] 

['BadGuy', 'MainlyNice', 'MainlyNice', 'MainlyBad', 'MainlyBad', 'MainlyBad', 'TitForTat', 'TitForTat', 'TitForTat', 'TitForTat', 'ResentfulGuy', 'ResentfulGuy', 'ResentfulGuy', 'ResentfulGuy', 'ResentfulGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'TrustingGuy', 'Thanos', 'Thanos', 'Thanos', 'Thanos', 'Thanos', 'Thanos', 'Thanos']
Are the two methods equivalent?  True
